### establish a percent change metric
rolling forward

In [1]:
!pip3 install matplotlib
!pip3 install altair

In [2]:
import altair as alt
import pandas as pd
import os
import numpy as np

In [3]:
!mv $(find . -type d -name "lob_caps" -exec grep -q MATCH {} \; -print0 | xargs -0 echo) backup_match/

grep: ./lob_caps: Is a directory
usage: mv [-f | -i | -n] [-hv] source target
       mv [-f | -i | -n] [-v] source ... directory


In [4]:
#https://stackoverflow.com/a/21232849 model 
def getCAPSByDateAndType(type):  #returns a dict, date + df caps for that date, then extended date and time
                                # print("for type, ", type)
    ret = []
    for root, dirs, files in os.walk("./lob_caps/"):
        for filename in files:
            if type in filename:
#                 print("CAPS file, ", filename) #mac, do find . -name ._\* -delete
                ret.append(filename)
    return ret

csvFileList = getCAPSByDateAndType("CAPS") #iterate this array to dip into each csv, later on
li = []                         #form the endFrame / global data frame around this array
for filename in csvFileList:
    csv = "lob_caps/" + filename
    # print(csv)
    df = pd.read_csv(csv, index_col=None, header=0)
    li.append(df)

capsFrame = pd.concat(li, axis=0, ignore_index=True) #end frame contains all data
capsFrame.sort_values(by=['time'], ascending=True)   #sorted by time into one time series
print("for new df: ", capsFrame.shape[0])
start = capsFrame["time"].min()
end = capsFrame["time"].max()
print("start: ", start, " end: ", end)
print(capsFrame.columns)

for new df:  284997
start:  1660221600292.0  end:  1691444880101.0
Index(['bc', 'ac', 'tbv', 'tav', 'time', 'mp', 'minBid'], dtype='object')


In [5]:
def getSKEWByDateAndType(type):  #returns a dict, date + df caps for that date, then extended date and time
                                # print("for type, ", type)
    ret = []
    for root, dirs, files in os.walk("./lob_caps/"):
        for filename in files:
            if type in filename:
#                 print("CAPS file, ", filename) #mac, do find . -name ._\* -delete
                ret.append(filename)
    return ret

csvFileList = getSKEWByDateAndType("MEANSHIFT") #iterate this array to dip into each csv, later on
li = []                         #form the endFrame / global data frame around this array
for filename in csvFileList:
    csv = "lob_caps/" + filename
    # print(csv)
    df = pd.read_csv(csv, index_col=None, header=0)
    li.append(df)

skewFrame = pd.concat(li, axis=0, ignore_index=True) #end frame contains all data
skewFrame.sort_values(by=['timeStamp'], ascending=True)   #sorted by time into one time series
skewFrame.rename(columns={'timeStamp': 'time'}, inplace=True)
print("for new df: ", skewFrame.shape[0])
start = skewFrame["time"].min()
end = skewFrame["time"].max()
print("start: ", start, " end: ", end)
print(skewFrame.columns)

for new df:  139237
start:  1660221606227  end:  1691444966359
Index(['mp', 'time', 'mean', 'skew'], dtype='object')


In [6]:
# Merging those two data frames Will not take place based on a Shared key of time
# You'll need to look up the SKU value for every row based on approximation
# merged_df = pd.merge(capsFrame, skewFrame, on='time')

In [7]:
skewFrame.head(10)

,mp,time,mean,skew
0,17.66,1677651424290,5877.906080,-5860.246080
1,17.66,1677651427404,5906.691222,-5889.031222
2,17.65,1677651430373,5944.892609,-5927.242609
3,17.66,1677651436323,5950.952405,-5933.292405
4,17.66,1677651439323,5926.797221,-5909.137221
5,17.66,1677651442399,5909.721180,-5892.061180
6,17.66,1677651445381,5898.866475,-5881.206475
7,17.66,1677651448534,5899.116796,-5881.456796
8,17.66,1677651451564,5891.509682,-5873.849682
9,17.66,1677651454328,5891.185193,-5873.525193


### Apply skew and mean values from mean shift to the caps data frame

for the data frame capsFrame set the column of 'mean' equal to the data frame, skewFrame 'mean' where the 'time' column in both data frames is approximately equal to within one minute



In [8]:
# capsFrame.loc[capsFrame['time'].sub(skewFrame['time'].values[0]).abs().idxmin(), 'mean'] = skewFrame['mean'].values[0]
# capsFrame.loc[capsFrame['time'].sub(skewFrame['time'].values[0]).abs().idxmin(), 'skew'] = skewFrame['skew'].values[0]
# capsFrame.loc[capsFrame['time'].sub(skewFrame['time'].values[0]).abs().lt(pd.Timedelta(minutes=2)).idxmax(), 'mean'] = skewFrame['mean'].values[0]
# 
# Could not get this operation to work try it again using a range of values once the precursor and surge are defined


In [9]:
capsFrame.head(2)

,bc,ac,tbv,tav,time,mp,minBid
0,2435138.24,10557430.61,493817.73,244833.13,1.672418e+12,10.78,10.74
1,2430127.12,10557235.51,493352.24,244815.55,1.672418e+12,10.78,10.74


In [10]:


# Load your time series data into a pandas dataframe
caps_df = capsFrame   # pd.read_csv('Fri Apr 07 2023 12:26:51 GMT-0700 (Pacific Daylight Time)-CAPS.csv')
lookback_period = 10 # in rows
caps_df['change'] = caps_df['mp'].pct_change(periods=lookback_period)
# caps_df.sample
print(caps_df.shape[0], caps_df.columns)# Calculate the returns of your asset over a fixed lookback period

284997 Index(['bc', 'ac', 'tbv', 'tav', 'time', 'mp', 'minBid', 'change'], dtype='object')


In [11]:
#for period, average or mean change metric. this changes with window size
meanChange = round(caps_df['change'].mean(),8)
meanChange

0.00046625

In [12]:
#reset mean chang ebery 24 hrs

In [13]:
# identify units of 10 rows where the percent change is greater or less than the threshold

threshold = meanChange
surges = []
precursors = []

for i in range(0,len(caps_df),10):

    if caps_df.iloc[i:i+10]['change'].mean() >= threshold:
        
        surges.append({'time': caps_df.iloc[i]['time'],
                       's_MP': caps_df.iloc[i]['mp'],
                       'change': caps_df.iloc[i:i+10]['change'].mean(),
                       'type':'surge'})  #['bc', 'ac', 'tbv', 'tav', 'time', 'mp', 'minBid', 'change']
    else:
        precursors.append({'time': caps_df.iloc[i]['time'],
                           'p_MP': caps_df.iloc[i]['mp'],
                           'change': caps_df.iloc[i:i+10]['change'].mean(),
                            'type':'precursor',
                            'p_buyCap':caps_df.iloc[i]['bc'], 
                            'p_askCap':caps_df.iloc[i]['ac'],
                            'p_totalBidVol':caps_df.iloc[i]['tbv'],
                            'p_totalAskVol':caps_df.iloc[i]['tav']
                            })  

In [1]:
# for item in surges:
#     print(item)

In [2]:
# for item in precursors:
#     print(item)

In [16]:
surges_df = pd.DataFrame(surges)
precursors_df = pd.DataFrame(precursors)
sequence_df = pd.concat([surges_df, precursors_df]).sort_values(by=['time'], ascending=True)

In [17]:
sequence_df.index

Int64Index([ 7572, 13111, 13112, 13113, 13114, 13115, 13116,  7573, 13117,
             7574,
            ...
             1057,  1058,   661,  1059,   662,  1060,  1061,   663,  1062,
              664],
           dtype='int64', length=28500)

In [18]:
# for index, row in sequence_df.iterrows():
#     print(row['surge'], row['precursor'])
sequence_df['type'].head(40)

7572         surge
13111    precursor
13112    precursor
13113    precursor
13114    precursor
13115    precursor
13116    precursor
7573         surge
13117    precursor
7574         surge
7575         surge
13118    precursor
13119    precursor
13120    precursor
13121    precursor
7576         surge
13122    precursor
13123    precursor
7577         surge
13124    precursor
13125    precursor
13126    precursor
13127    precursor
13128    precursor
13129    precursor
13130    precursor
7578         surge
7579         surge
13131    precursor
13132    precursor
7580         surge
7581         surge
7582         surge
7583         surge
13133    precursor
13134    precursor
13135    precursor
13136    precursor
7584         surge
13137    precursor
Name: type, dtype: object

In [3]:
# sequence_df.head(45)

In [20]:

# line = alt.Chart(sequence_df).mark_line(color='green').encode(
#     x='time',
#     y='s_MP'
# )

# s_bar = alt.Chart(sequence_df).mark_bar().encode(
#     x='time',
#     y='s_MP',
#     color='type:N'
# )

# p_bar = alt.Chart(sequence_df).mark_bar().encode(
#     x='time',
#     y='p_MP',
#     color='type:N'
# )

# chart = (line + s_bar + p_bar).properties(width=600, height=500)
# chart.title = 'Sequential order of precursor and surges for April 7th 2023'
# subtitle = 'Precursors are contiguous periods where percentage rate of growth is less than threshold'
# chart.properties(title=alt.TitleParams(text=[chart.title, subtitle], baseline='bottom', orient='top', anchor='start', fontSize=14))
# chart

In [21]:
sequence_df.columns

Index(['time', 's_MP', 'change', 'type', 'p_MP', 'p_buyCap', 'p_askCap',
       'p_totalBidVol', 'p_totalAskVol'],
      dtype='object')

### Grouping together bucketed surge items

Create one line to describe a precursor or search and it's related order book statistics

In [22]:

sequence_df['group'] = (sequence_df['type'] != sequence_df['type'].shift(1)).cumsum()
sequence_df['length'] = sequence_df.groupby(['type', 'group'])['group'].transform('count')
# sequence_df['identifier'] = sequence_df.groupby(['type', 'group'])['time'].transform('min') #prep the label early, if surge?
print(sequence_df.shape[0])
sequence_df['sum_change'] = sequence_df.groupby(['type', 'group'])['change'].transform('sum')
print(sequence_df.shape[0])

sequence_df['area']  = sequence_df.apply(lambda row: row['length'] * row['sum_change'], axis=1)
sequence_df.loc[sequence_df['type'] == 'surge', 'surge_area'] = sequence_df.loc[sequence_df['type'] == 'surge', 'area']
sequence_df.columns
# sequence_df['area'] = sequence_df.groupby(['type', 'group'])['sum_change'].multiply(sequence_df['length'])
# sequence_df.loc[sequence_df['type'] == 'surge', 'surge_area'] =  sequence_df['length'] * sequence_df['sum_change']


# sequence_df['sum'] = sequence_df.groupby(['surge', 'group'])['change'].transform('sum')

# sequence_df['end_time'] = sequence_df.groupby(['surge', 'group'])['time'].transform('max')
# sequence_df['type'] = sequence_df['surge']

# sequence_df['buyCapSum'] = sequence_df.groupby(['surge', 'group'])['buyCap'].transform('avg')
# sequence_df['askCapSum'] = sequence_df.groupby(['surge', 'group'])['askCap'].transform('avg')

# calculate the area for the surge

# sequence_df = sequence_df.drop('next_value', axis=1)
# sequence_df.loc[sequence_df['bucket'] == 'surge', 'surge_length'] =  sequence_df['length']
# sequence_df.drop('length', axis=1, inplace=True)
# df = df.loc[:,~df.columns.duplicated()]

#unique_df = sequence_df.groupby('identifier').first().reset_index()

# unique_df.loc[unique_df['surge'] == '1', 'surge_length'] = unique_df['length']
# unique_df.loc[unique_df['surge'] == '0', 'length'] = 0

28500
28500


Index(['time', 's_MP', 'change', 'type', 'p_MP', 'p_buyCap', 'p_askCap',
       'p_totalBidVol', 'p_totalAskVol', 'group', 'length', 'sum_change',
       'area', 'surge_area'],
      dtype='object')

In [23]:
sequence_df.head(20)

,time,s_MP,change,type,p_MP,p_buyCap,p_askCap,p_totalBidVol,p_totalAskVol,group,length,sum_change,area,surge_area
7572,1.660222e+12,29.99,0.317588,surge,NaN,NaN,NaN,NaN,NaN,1,1,0.317588,0.317588,0.317588
13111,1.660222e+12,NaN,-0.000467,precursor,30.00,7349599.51,1.229796e+09,716681.93,314676.40,2,6,-0.004943,-0.029656,NaN
13112,1.660222e+12,NaN,-0.000900,precursor,29.99,7317126.26,1.229765e+09,715480.59,313616.24,2,6,-0.004943,-0.029656,NaN
13113,1.660222e+12,NaN,-0.001135,precursor,29.93,7315988.33,1.229753e+09,715534.43,313253.94,2,6,-0.004943,-0.029656,NaN
13114,1.660222e+12,NaN,-0.001170,precursor,29.90,7342367.97,1.229765e+09,716464.60,313645.49,2,6,-0.004943,-0.029656,NaN
13115,1.660222e+12,NaN,-0.000067,precursor,29.88,7312475.73,1.229757e+09,715475.00,313414.67,2,6,-0.004943,-0.029656,NaN
13116,1.660222e+12,NaN,-0.001205,precursor,29.86,7267312.95,1.229812e+09,713921.46,315283.24,2,6,-0.004943,-0.029656,NaN
7573,1.660222e+12,29.85,0.000469,surge,NaN,NaN,NaN,NaN,NaN,3,1,0.000469,0.000469,0.000469
13117,1.660222e+12,NaN,0.000402,precursor,29.90,7307989.88,1.229792e+09,715352.87,314642.84,4,1,0.000402,0.000402,NaN
7574,1.660222e+12,29.87,0.001105,surge,NaN,NaN,NaN,NaN,NaN,5,2,0.001607,0.003214,0.003214


## Critical group by unique identifier

In [24]:
unique_df = sequence_df.groupby('group').first().reset_index()
print(unique_df)

       group          time   s_MP    change       type   p_MP    p_buyCap  \
0          1  1.660222e+12  29.99  0.317588      surge    NaN         NaN   
1          2  1.660222e+12    NaN -0.000467  precursor  30.00  7349599.51   
2          3  1.660222e+12  29.85  0.000469      surge    NaN         NaN   
3          4  1.660222e+12    NaN  0.000402  precursor  29.90  7307989.88   
4          5  1.660222e+12  29.87  0.001105      surge    NaN         NaN   
...      ...           ...    ...       ...        ...    ...         ...   
11136  11137  1.691442e+12  12.44  0.000564      surge    NaN         NaN   
11137  11138  1.691442e+12    NaN -0.001848  precursor  12.43  1210140.47   
11138  11139  1.691443e+12  12.41  0.002981      surge    NaN         NaN   
11139  11140  1.691444e+12    NaN -0.000720  precursor  12.46  1227623.76   
11140  11141  1.691445e+12  12.43  0.002744      surge    NaN         NaN   

           p_askCap  p_totalBidVol  p_totalAskVol  length  sum_change  \
0 

### Merge even and odd Rows

In [25]:
even_df = unique_df.iloc[::2].reset_index(drop=True)
odd_df = unique_df.iloc[1::2].reset_index(drop=True)

merged_df = pd.concat([even_df, odd_df], axis=1)

print(merged_df)

      group          time   s_MP    change   type  p_MP  p_buyCap  p_askCap  \
0         1  1.660222e+12  29.99  0.317588  surge   NaN       NaN       NaN   
1         3  1.660222e+12  29.85  0.000469  surge   NaN       NaN       NaN   
2         5  1.660222e+12  29.87  0.001105  surge   NaN       NaN       NaN   
3         7  1.660222e+12  29.89  0.001610  surge   NaN       NaN       NaN   
4         9  1.660223e+12  29.90  0.001372  surge   NaN       NaN       NaN   
...     ...           ...    ...       ...    ...   ...       ...       ...   
5566  11133  1.691438e+12  12.43  0.000805  surge   NaN       NaN       NaN   
5567  11135  1.691441e+12  12.43  0.001530  surge   NaN       NaN       NaN   
5568  11137  1.691442e+12  12.44  0.000564  surge   NaN       NaN       NaN   
5569  11139  1.691443e+12  12.41  0.002981  surge   NaN       NaN       NaN   
5570  11141  1.691445e+12  12.43  0.002744  surge   NaN       NaN       NaN   

      p_totalBidVol  p_totalAskVol  ...       type 

In [26]:
nan_cols = merged_df.dropna(axis=1, how='all')
nan_cols.head()

,group,time,s_MP,change,type,length,sum_change,area,surge_area,group,...,change,type,p_MP,p_buyCap,p_askCap,p_totalBidVol,p_totalAskVol,length,sum_change,area
0,1,1.660222e+12,29.99,0.317588,surge,1,0.317588,0.317588,0.317588,2.0,...,-0.000467,precursor,30.00,7349599.51,1.229796e+09,716681.93,314676.40,6.0,-0.004943,-0.029656
1,3,1.660222e+12,29.85,0.000469,surge,1,0.000469,0.000469,0.000469,4.0,...,0.000402,precursor,29.90,7307989.88,1.229792e+09,715352.87,314642.84,1.0,0.000402,0.000402
2,5,1.660222e+12,29.87,0.001105,surge,2,0.001607,0.003214,0.003214,6.0,...,-0.000066,precursor,29.92,7285244.89,1.229754e+09,714476.10,313575.99,4.0,-0.002773,-0.011090
3,7,1.660222e+12,29.89,0.001610,surge,1,0.001610,0.001610,0.001610,8.0,...,-0.000435,precursor,29.88,7318707.15,1.229744e+09,715730.56,313546.58,2.0,-0.000200,-0.000400
4,9,1.660223e+12,29.90,0.001372,surge,1,0.001372,0.001372,0.001372,10.0,...,-0.000534,precursor,29.93,7280582.12,1.229794e+09,714140.13,314889.39,7.0,-0.013670,-0.095690


In [33]:
nan_cols.columns

Index(['group', 'time', 's_MP', 'change', 'type', 'length', 'sum_change',
       'area', 'surge_area', 'group', 'time', 'change', 'type', 'p_MP',
       'p_buyCap', 'p_askCap', 'p_totalBidVol', 'p_totalAskVol', 'length',
       'sum_change', 'area'],
      dtype='object')

### Write to CSV: step one, pipeline
Label to use is surge_area

In [34]:
# nan_cols = nan_cols.rename(columns={'group': 'group_1', 'time': 'time_1', 'change': 'change_1', 'type': 'type_1', 'length': 'length_1', 'sum_change': 'sum_change_1', 'area': 'area_1'})
writeable_df = nan_cols['group', 'time', 's_MP', 'change', 'type', 'length', 'sum_change','area', \
                        'surge_area', 'group', 'time', 'change', 'type', 'p_MP',
                       'p_buyCap', 'p_askCap', 'p_totalBidVol', 'p_totalAskVol', 'length','sum_change', 'area']

writeable_df.columns = ['group', 'time', 's_MP', 'change', 'type', 'length', 'sum_change',
       's_area', 'surge_area', 'p_group', 'p_time', 'p_change', 'p_type', 'p_MP',
       'p_buyCap', 'p_askCap', 'p_totalBidVol', 'p_totalAskVol', 'p_length',
       'p_sum_change', 'p_area']

# df.to_csv('filename.csv', index=False)
writeable_df.to_csv('pipeline1.csv', index=False)


In [29]:
'''The repeating elements in the list are:
- group
- time
- change
- type
- length
- sum_change
- area '''

'The repeating elements in the list are:\n- group\n- time\n- change\n- type\n- length\n- sum_change\n- area '